# Classificando Áudios

Vamos agora ver os dados do Dataset Fleurs, do Google: https://huggingface.co/datasets/google/fleurs.

Este Dataset contém frases faladas em diferentes línguas, junto de sua transcrição.

São **mais de 330 GB de dados**, portanto vamos usar o modo de streaming de dados do Hugging Face para não precisar baixar tudo (o que sequer cabe em memória):

## Baixando dados no modo streaming

In [1]:
from pathlib import Path

pasta_saida = Path('audios') / 'vozes'
pasta_saida.mkdir(exist_ok=True)

In [2]:
from datasets import load_dataset

nome_dataset = "google/fleurs"
lingua_dataset = "pt_br"
dados = load_dataset(nome_dataset, name=lingua_dataset, split='train', streaming=True)
dados

C:\Users\Juliano\Desktop\IAs para Imagens e Áudio com Hugging Face\venv\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


IterableDataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    n_shards: 1
})

## Observando os dados

In [3]:
primeiras_linhas = dados.take(5)
primeiras_linhas

IterableDataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    n_shards: 1
})

In [4]:
for linha in primeiras_linhas:
    print(linha)

{'id': 114, 'num_samples': 312960, 'path': None, 'audio': {'path': 'train/10009971053374752024.wav', 'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       9.54866409e-05, 1.48892403e-04, 1.79767609e-04]), 'sampling_rate': 16000}, 'transcription': 'o governo anterior considerado conservador da austrália se negou a ratificar kyoto afirmando que seria ruim para a economia por sua forte dependência das exportações de carvão ao mesmo tempo que alguns países como índia e china não tinham limitações por metas de emissões', 'raw_transcription': 'O governo anterior, considerado conservador, da Austrália se negou a ratificar Kyoto, afirmando que seria ruim para a economia por sua forte dependência das exportações de carvão, ao mesmo tempo que alguns países como Índia e China não tinham limitações por metas de emissões.', 'gender': 0, 'lang_id': 76, 'language': 'Portuguese', 'lang_group_id': 0}
{'id': 916, 'num_samples': 257280, 'path': None, 'audio': {'path': 'train/1001007

In [5]:
import IPython
import soundfile

for i, linha in enumerate(primeiras_linhas, 1):
    lingua = linha['language']
    dados_som = linha['audio']['array']
    taxa_amostragem = linha['audio']['sampling_rate']
    caminho_saida = pasta_saida / f'{lingua}_{i:02d}.wav'
    # Salvando em um arquivo de áudio
    soundfile.write(file=caminho_saida, data=dados_som, samplerate=taxa_amostragem)
    # Exibindo no Jupyter Notebook
    display(IPython.display.Audio(data=dados_som, rate=taxa_amostragem))
    print(linha['transcription'])

o governo anterior considerado conservador da austrália se negou a ratificar kyoto afirmando que seria ruim para a economia por sua forte dependência das exportações de carvão ao mesmo tempo que alguns países como índia e china não tinham limitações por metas de emissões


alguns dos aplicativos desta categoria podem inclusive traduzir textos em línguas estrangeiras em placas ou outros objetos do mundo real quando o usuário aponta o smartphone para esses objetos


o ônibus da banda estava seguindo para o six flags st louis em missouri para tocar para várias pessoas num evento onde todos os ingressos estavam esgotados


por alguns trocados algumas crianças lhe contarão a história


esta é uma prática comum em outras partes do reino unido mas a justiça da escócia atua de maneira distinta e as cortes consideram a publicação de fotos como algo potencialmente prejudicial


# Modelos de IA para classificação de áudio

Vamos agora testar alguns modelos de **classificação de áudio**.

Esses modelos são treinados para classificar áudio dentro de algum contexto, como:
- dizer qual língua está sendo falada.
- dizer de qual espécie de ave é um barulho.
- dizer qual objeto produz o som.

O primeiro exemplo é um modelo de IA que diz qual a língua falada: https://huggingface.co/sanchit-gandhi/whisper-medium-fleurs-lang-id

In [6]:
from transformers import pipeline

modelo = 'sanchit-gandhi/whisper-medium-fleurs-lang-id'
classificador = pipeline('audio-classification', model=modelo)
classificador

In [ ]:
classificador.feature_extractor.sampling_rate

Este modelo possui o mesmo sampling rate que nossos exemplos do Dataset, portanto, não é preciso ajustar o sampling rate dos nossos dados. 

## Testando o modelo

In [ ]:
pasta_entrada = Path('audios') / 'vozes'
arquivos = sorted(pasta_entrada.iterdir())
arquivos

In [ ]:
for arquivo in arquivos:
    dados_som, taxa_amostragem = soundfile.read(file=arquivo)
    predicao = classificador(dados_som)
    print(predicao)
    print('-----')

**Sucesso!** Vamos testar com outra língua (agora montando um script completo, indo desde o momento de baixar os dados):

In [10]:
from datasets import load_dataset
from transformers import pipeline

nome_dataset = "google/fleurs"
lingua_dataset = "en_us"
dados = load_dataset(nome_dataset, name=lingua_dataset, split='train', streaming=True)

modelo = 'sanchit-gandhi/whisper-medium-fleurs-lang-id'
classificador = pipeline('audio-classification', model=modelo)

for linha in dados.take(5):
    predicao = classificador(linha["audio"]["array"])
    print(predicao)
    print('-----')

C:\Users\Juliano\Desktop\IAs para Imagens e Áudio com Hugging Face\venv\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


[{'score': 0.9999545812606812, 'label': 'English'}, {'score': 2.6251766485074768e-06, 'label': 'Norwegian'}, {'score': 2.0163679437246174e-06, 'label': 'Indonesian'}, {'score': 1.8816407418853487e-06, 'label': 'Swahili'}, {'score': 1.5627134644091711e-06, 'label': 'Kazakh'}]
-----
[{'score': 0.9999525547027588, 'label': 'English'}, {'score': 4.006352810392855e-06, 'label': 'Norwegian'}, {'score': 3.4527008665463654e-06, 'label': 'Kazakh'}, {'score': 2.4740352273511235e-06, 'label': 'Icelandic'}, {'score': 2.252613057862618e-06, 'label': 'Estonian'}]
-----
[{'score': 0.9999405145645142, 'label': 'English'}, {'score': 2.8429362828319427e-06, 'label': 'Kazakh'}, {'score': 2.7181481527804863e-06, 'label': 'Norwegian'}, {'score': 2.364858346481924e-06, 'label': 'Indonesian'}, {'score': 2.14833607969922e-06, 'label': 'Thai'}]
-----
[{'score': 0.999945878982544, 'label': 'English'}, {'score': 3.835805273411097e-06, 'label': 'Kazakh'}, {'score': 2.9356674531300087e-06, 'label': 'Norwegian'}, {

# Teste ao vivo

Vamos agora testar com áudio gravado do microfone do computador. Para isso, vamos precisar instalar mais uma biblioteca:

```bash
pip install sounddevice
```

Em sistemas UNIX (Mac / Linux) pode ser necessário instalar a [biblioteca PortAudio](https://files.portaudio.com/download.html) manualmente. Por exemplo, em Linux podemos instalá-la com `apt`:

```bash
sudo apt install libportaudio2
```

In [7]:
import sounddevice as sd

duracao = 10
taxa_amostragem = 16000
tamanho_vetor = int(duracao * taxa_amostragem)

gravacao = sd.rec(tamanho_vetor, samplerate=taxa_amostragem, channels=1)
sd.wait()

gravacao

array([[-0.0017395 ],
       [-0.00204468],
       [ 0.00039673],
       ...,
       [ 0.02398682],
       [ 0.02633667],
       [ 0.02648926]], dtype=float32)

In [8]:
type(gravacao)

numpy.ndarray

Para exibir o áudio, precisamos ajustar o formato do vetor para a horizontal (ele "empilha" os valores em colunas pois pode haver múltiplos canais de áudio na mesma gravação):

In [9]:
print(gravacao.shape)
gravacao = gravacao.ravel()
print(gravacao.shape)

(160000, 1)
(160000,)


In [10]:
IPython.display.Audio(data=gravacao, rate=taxa_amostragem)

E agora vamos testar no classificador:

In [11]:
classificador(gravacao)

[{'score': 0.9999797344207764, 'label': 'Portuguese'},
 {'score': 8.514022738381755e-06, 'label': 'Kabuverdianu'},
 {'score': 9.232427942151844e-07, 'label': 'Persian'},
 {'score': 7.92350590472779e-07, 'label': 'French'},
 {'score': 6.599144626306952e-07, 'label': 'Welsh'}]

Sucesso!

# Classificação *zero-shot*

Com alguns modelos, podemos fazer classificação *zero-shot*. Isso significa que nós mesmos podemos passar as classes para o modelo classificar dentre elas.

Vamos ver um exemplo simples com o mesmo Dataset visto na aula anterior: https://huggingface.co/datasets/ashraq/esc50

In [12]:
dados = load_dataset('ashraq/esc50', split='train[:10]')  # Apenas as 10 primeiras linhas
for linha in dados:
    print(linha)

Repo card metadata block was not found. Setting CardData to empty.


{'filename': '1-100032-A-0.wav', 'fold': 1, 'target': 0, 'category': 'dog', 'esc10': True, 'src_file': 100032, 'take': 'A', 'audio': {'path': None, 'array': array([0., 0., 0., ..., 0., 0., 0.]), 'sampling_rate': 44100}}
{'filename': '1-100038-A-14.wav', 'fold': 1, 'target': 14, 'category': 'chirping_birds', 'esc10': False, 'src_file': 100038, 'take': 'A', 'audio': {'path': None, 'array': array([-0.01184082, -0.10336304, -0.14141846, ...,  0.06985474,
        0.04049683,  0.00274658]), 'sampling_rate': 44100}}
{'filename': '1-100210-A-36.wav', 'fold': 1, 'target': 36, 'category': 'vacuum_cleaner', 'esc10': False, 'src_file': 100210, 'take': 'A', 'audio': {'path': None, 'array': array([-0.00695801, -0.01251221, -0.01126099, ...,  0.215271  ,
       -0.00875854, -0.28903198]), 'sampling_rate': 44100}}
{'filename': '1-100210-B-36.wav', 'fold': 1, 'target': 36, 'category': 'vacuum_cleaner', 'esc10': False, 'src_file': 100210, 'take': 'B', 'audio': {'path': None, 'array': array([0.53897095, 

In [13]:
modelo = 'laion/clap-htsat-fused'
classificador = pipeline("zero-shot-audio-classification", model=modelo)
print(classificador.feature_extractor.sampling_rate)

config.json:   0%|          | 0.00/5.42k [00:00<?, ?B/s]

C:\Users\Juliano\Desktop\IAs para Imagens e Áudio com Hugging Face\venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Juliano\.cache\huggingface\hub\models--laion--clap-htsat-fused. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

48000


In [14]:
from datasets import Audio

dados = dados.cast_column("audio", Audio(sampling_rate=48000))

In [15]:
labels = [
    'Sound of a dog',
    'Sound of birds',
    'Sound of a cat',
    'Sound of a cow',
    'Sound of a vacuum cleaner'
]

for linha in dados:
    categoria = linha['category']
    predicao = classificador(linha['audio']['array'], candidate_labels=labels)
    print(f'Tipo: {categoria}\nPredição: {predicao[0]["label"]} (score: {predicao[0]["score"]})')
    print('-----')

Tipo: dog
Predição: Sound of a dog (score: 0.9988256096839905)
-----
Tipo: chirping_birds
Predição: Sound of birds (score: 0.9992444515228271)
-----
Tipo: vacuum_cleaner
Predição: Sound of a vacuum cleaner (score: 0.9987397789955139)
-----
Tipo: vacuum_cleaner
Predição: Sound of a vacuum cleaner (score: 0.9998911619186401)
-----
Tipo: thunderstorm
Predição: Sound of a cow (score: 0.8125988841056824)
-----
Tipo: thunderstorm
Predição: Sound of a cow (score: 0.8609119653701782)
-----
Tipo: door_wood_knock
Predição: Sound of a cow (score: 0.980539083480835)
-----
Tipo: can_opening
Predição: Sound of a dog (score: 0.8917919397354126)
-----
Tipo: crow
Predição: Sound of birds (score: 0.7291918396949768)
-----
Tipo: door_wood_knock
Predição: Sound of a cow (score: 0.6846059560775757)
-----


Modelos desse tipo poderiam ser usados para diversas outras aplicações: 
- Comandos de voz (Ex: "Ok Google", "Alexa")
- Detectar padrões sonoros (respiração de pacientes em UTI, sons na rua para segurança pública, ...)